In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5247673407502333134, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16056669969009267512
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [9]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [13]:
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
# from keras.applications.xception import Xception, preprocess_input
# from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

bottleneck_final_model=NASNetLarge(weights='imagenet', include_top=False, pooling = 'avg', input_tensor=Input(shape=input_shape))

In [14]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "NASNetLarge_descriptors"

In [15]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [16]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [17]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [18]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [19]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [20]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [21]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [22]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [23]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'lr': 1e-06, 'beta_2': 0.7, 'beta_1': 0.9}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.7862 - acc: 0.2080 - val_loss: 1.7590 - val_acc: 0.2956
Epoch 2/15
 - 4s - loss: 1.6673 - acc: 0.3147 - val_loss: 1.7264 - val_acc: 0.3513
Epoch 3/15
 - 4s - loss: 1.5923 - acc: 0.3796 - val_loss: 1.6962 - val_acc: 0.3678
Epoch 4/15
 - 4s - loss: 1.5339 - acc: 0.4243 - val_loss: 1.6678 - val_acc: 0.3559
Epoch 5/15
 - 4s - loss: 1.4882 - acc: 0.4631 - val_loss: 1.6412 - val_acc: 0.3492
Epoch 6/15
 - 4s - loss: 1.4490 - acc: 0.4847 - val_loss: 1.6168 - val_acc: 0.3456
Epoch 00006: early stopping
14958/14958 [==============================] - 1s 41us/step
Validation Accuracy: 34.5568%
Validation Loss: 1.616782413763961
Test Accuracy: 0.48166151126822804
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 0.0001, 'beta_2': 0.

Epoch 5/15
 - 4s - loss: 0.5804 - acc: 0.8139 - val_loss: 0.9763 - val_acc: 0.6764
Epoch 6/15
 - 4s - loss: 0.5744 - acc: 0.8180 - val_loss: 0.9909 - val_acc: 0.6803
Epoch 7/15
 - 4s - loss: 0.5562 - acc: 0.8244 - val_loss: 0.9832 - val_acc: 0.6833
Epoch 8/15
 - 4s - loss: 0.5548 - acc: 0.8280 - val_loss: 0.9940 - val_acc: 0.6860
Epoch 9/15
 - 4s - loss: 0.5454 - acc: 0.8325 - val_loss: 1.0019 - val_acc: 0.6889
Epoch 10/15
 - 4s - loss: 0.5377 - acc: 0.8348 - val_loss: 1.0245 - val_acc: 0.6905
Epoch 11/15
 - 4s - loss: 0.5409 - acc: 0.8369 - val_loss: 1.0342 - val_acc: 0.6898
Epoch 12/15
 - 4s - loss: 0.5391 - acc: 0.8393 - val_loss: 1.0330 - val_acc: 0.6911
Epoch 13/15
 - 4s - loss: 0.5352 - acc: 0.8434 - val_loss: 1.0526 - val_acc: 0.6931
Epoch 14/15
 - 4s - loss: 0.5348 - acc: 0.8430 - val_loss: 1.0816 - val_acc: 0.6902
Epoch 15/15
 - 4s - loss: 0.5426 - acc: 0.8452 - val_loss: 1.0737 - val_acc: 0.6945
14958/14958 [==============================] - 1s 42us/step
Validation Accuracy: 

Epoch 00005: early stopping
14958/14958 [==============================] - 1s 43us/step
Validation Accuracy: 65.9580%
Validation Loss: 1.7889786313419502
Test Accuracy: 0.7655766681396377
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 1e-07, 'beta_2': 0.6, 'beta_1': 0.6}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.8063 - acc: 0.1888 - val_loss: 1.7713 - val_acc: 0.1964
Epoch 2/15
 - 4s - loss: 1.7953 - acc: 0.1992 - val_loss: 1.7681 - val_acc: 0.2008
Epoch 3/15
 - 4s - loss: 1.7868 - acc: 0.2040 - val_loss: 1.7650 - val_acc: 0.2050
Epoch 4/15
 - 4s - loss: 1.7725 - acc: 0.2166 - val_loss: 1.7619 - val_acc: 0.2101
Epoch 5/15
 - 4s - loss: 1.7634 - acc: 0.2270 - val_loss: 1.7589 - val_acc: 0.2129
Epoch 6/15
 - 4s - loss: 1.7530 - acc: 0.2348 - val_loss: 1.7559 - val_acc: 0.2161
Epoch 7/15
 - 4s - loss: 1.7449 - acc: 0.2445 - 

Parameters testing:  {'lr': 1e-07, 'beta_2': 0.995, 'beta_1': 0.9}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 6s - loss: 1.8450 - acc: 0.1671 - val_loss: 1.7972 - val_acc: 0.1761
Epoch 2/15
 - 5s - loss: 1.8336 - acc: 0.1747 - val_loss: 1.7947 - val_acc: 0.1978
Epoch 3/15
 - 5s - loss: 1.8248 - acc: 0.1775 - val_loss: 1.7924 - val_acc: 0.2186
Epoch 4/15
 - 5s - loss: 1.8102 - acc: 0.1887 - val_loss: 1.7900 - val_acc: 0.2370
Epoch 5/15
 - 5s - loss: 1.8013 - acc: 0.1998 - val_loss: 1.7877 - val_acc: 0.2541
Epoch 6/15
 - 5s - loss: 1.7906 - acc: 0.2112 - val_loss: 1.7854 - val_acc: 0.2686
Epoch 7/15
 - 5s - loss: 1.7785 - acc: 0.2206 - val_loss: 1.7833 - val_acc: 0.2801
Epoch 8/15
 - 5s - loss: 1.7671 - acc: 0.2319 - val_loss: 1.7812 - val_acc: 0.2926
Epoch 9/15
 - 5s - loss: 1.7600 - acc: 0.2404 - val_loss: 1.7791 - val_acc: 0.3002
Epoch 10/15
 - 5s - loss: 1.7516 - acc: 0.2505 - val_loss: 1.7770 - val_acc: 0.3059
Epoch 11/15
 - 5s - loss: 1.7439 - acc: 0.2615 - val

Test Accuracy: 0.7486375018412137
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 1e-05, 'beta_2': 0.995, 'beta_1': 0.8}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 6s - loss: 1.5478 - acc: 0.4165 - val_loss: 1.5567 - val_acc: 0.4017
Epoch 2/15
 - 5s - loss: 1.2898 - acc: 0.5794 - val_loss: 1.4267 - val_acc: 0.4799
Epoch 3/15
 - 5s - loss: 1.1511 - acc: 0.6456 - val_loss: 1.3309 - val_acc: 0.5243
Epoch 4/15
 - 5s - loss: 1.0562 - acc: 0.6817 - val_loss: 1.2572 - val_acc: 0.5562
Epoch 5/15
 - 5s - loss: 0.9877 - acc: 0.6974 - val_loss: 1.2011 - val_acc: 0.5748
Epoch 6/15
 - 5s - loss: 0.9319 - acc: 0.7083 - val_loss: 1.1561 - val_acc: 0.5911
Epoch 7/15
 - 5s - loss: 0.8838 - acc: 0.7238 - val_loss: 1.1186 - val_acc: 0.6015
Epoch 8/15
 - 5s - loss: 0.8464 - acc: 0.7324 - val_loss: 1.0856 - val_acc: 0.6117
Epoch 9/15
 - 5s - loss: 0.8157 - 

...

 - 24s - loss: 0.9259 - acc: 0.7015 - val_loss: 1.0277 - val_acc: 0.6407
Epoch 2/15
 - 13s - loss: 0.6340 - acc: 0.7915 - val_loss: 0.9594 - val_acc: 0.6715
Epoch 3/15
 - 13s - loss: 0.5576 - acc: 0.8150 - val_loss: 0.9674 - val_acc: 0.6782
Epoch 4/15
 - 11s - loss: 0.5210 - acc: 0.8296 - val_loss: 1.0042 - val_acc: 0.6768
Epoch 5/15
 - 11s - loss: 0.4924 - acc: 0.8384 - val_loss: 0.9778 - val_acc: 0.6920
Epoch 6/15
 - 11s - loss: 0.4741 - acc: 0.8451 - val_loss: 0.9821 - val_acc: 0.6906
Epoch 7/15
 - 12s - loss: 0.4569 - acc: 0.8524 - val_loss: 0.9854 - val_acc: 0.6861
Epoch 8/15
 - 11s - loss: 0.4413 - acc: 0.8579 - val_loss: 1.0661 - val_acc: 0.6939
Epoch 9/15
 - 12s - loss: 0.4278 - acc: 0.8620 - val_loss: 1.0742 - val_acc: 0.6878
Epoch 10/15
 - 12s - loss: 0.4225 - acc: 0.8654 - val_loss: 1.0581 - val_acc: 0.6978
Epoch 11/15
 - 12s - loss: 0.4181 - acc: 0.8697 - val_loss: 1.1039 - val_acc: 0.6918
Epoch 12/15
 - 12s - loss: 0.4034 - acc: 0.8728 - val_loss: 1.1395 - val_acc: 0.6909


Epoch 4/15
 - 12s - loss: 0.8490 - acc: 0.7420 - val_loss: 1.1144 - val_acc: 0.6225
Epoch 5/15
 - 11s - loss: 0.7862 - acc: 0.7569 - val_loss: 1.0684 - val_acc: 0.6340
Epoch 6/15
 - 13s - loss: 0.7336 - acc: 0.7644 - val_loss: 1.0422 - val_acc: 0.6398
Epoch 7/15
 - 10s - loss: 0.6947 - acc: 0.7774 - val_loss: 1.0078 - val_acc: 0.6483
Epoch 8/15
 - 12s - loss: 0.6620 - acc: 0.7833 - val_loss: 0.9946 - val_acc: 0.6516
Epoch 9/15
 - 13s - loss: 0.6381 - acc: 0.7890 - val_loss: 0.9921 - val_acc: 0.6552
Epoch 10/15
 - 13s - loss: 0.6225 - acc: 0.7940 - val_loss: 0.9558 - val_acc: 0.6631
Epoch 11/15
 - 11s - loss: 0.5973 - acc: 0.8010 - val_loss: 0.9761 - val_acc: 0.6609
Epoch 12/15
 - 11s - loss: 0.5827 - acc: 0.8068 - val_loss: 0.9570 - val_acc: 0.6666
Epoch 13/15
 - 12s - loss: 0.5667 - acc: 0.8104 - val_loss: 0.9795 - val_acc: 0.6672
Epoch 14/15
 - 11s - loss: 0.5506 - acc: 0.8160 - val_loss: 0.9474 - val_acc: 0.6719
Epoch 15/15
 - 12s - loss: 0.5384 - acc: 0.8163 - val_loss: 0.9566 - va

Epoch 14/15
 - 11s - loss: 0.3979 - acc: 0.8604 - val_loss: 0.8589 - val_acc: 0.6978
Epoch 15/15
 - 11s - loss: 0.3879 - acc: 0.8635 - val_loss: 0.8666 - val_acc: 0.6968
14958/14958 [==============================] - 2s 112us/step
Validation Accuracy: 69.6818%
Validation Loss: 0.8665884601681735
Test Accuracy: 0.8112387685962587
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 0.01, 'beta_2': 0.8, 'beta_1': 0.9}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 1.1701 - acc: 0.6877 - val_loss: 1.1196 - val_acc: 0.6723
Epoch 2/15
 - 11s - loss: 1.2000 - acc: 0.6807 - val_loss: 2.0343 - val_acc: 0.6195
Epoch 3/15
 - 10s - loss: 1.2883 - acc: 0.6789 - val_loss: 1.6173 - val_acc: 0.6503
Epoch 4/15
 - 12s - loss: 1.4546 - acc: 0.6740 - val_loss: 2.1077 - val_acc: 0.6566
Epoch 00004: early stopping
14958/14958 [===========================

 - 13s - loss: 0.6735 - acc: 0.7795 - val_loss: 1.0354 - val_acc: 0.6535
Epoch 4/15
 - 11s - loss: 0.6351 - acc: 0.7966 - val_loss: 1.0123 - val_acc: 0.6674
Epoch 5/15
 - 12s - loss: 0.6156 - acc: 0.8064 - val_loss: 1.0453 - val_acc: 0.6678
Epoch 6/15
 - 11s - loss: 0.6040 - acc: 0.8135 - val_loss: 1.0428 - val_acc: 0.6746
Epoch 7/15
 - 11s - loss: 0.5981 - acc: 0.8167 - val_loss: 1.0652 - val_acc: 0.6750
Epoch 8/15
 - 11s - loss: 0.5992 - acc: 0.8209 - val_loss: 1.0858 - val_acc: 0.6766
Epoch 9/15
 - 11s - loss: 0.5976 - acc: 0.8264 - val_loss: 1.0634 - val_acc: 0.6830
Epoch 10/15
 - 12s - loss: 0.5909 - acc: 0.8284 - val_loss: 1.1038 - val_acc: 0.6802
Epoch 11/15
 - 11s - loss: 0.5865 - acc: 0.8304 - val_loss: 1.1087 - val_acc: 0.6833
Epoch 12/15
 - 13s - loss: 0.5982 - acc: 0.8314 - val_loss: 1.1246 - val_acc: 0.6889
Epoch 13/15
 - 12s - loss: 0.5930 - acc: 0.8340 - val_loss: 1.1525 - val_acc: 0.6833
Epoch 14/15
 - 12s - loss: 0.5947 - acc: 0.8356 - val_loss: 1.1377 - val_acc: 0.688

Epoch 6/15
 - 11s - loss: 0.4591 - acc: 0.8499 - val_loss: 0.9878 - val_acc: 0.6860
Epoch 7/15
 - 13s - loss: 0.4447 - acc: 0.8548 - val_loss: 0.9494 - val_acc: 0.6968
Epoch 8/15
 - 13s - loss: 0.4216 - acc: 0.8611 - val_loss: 0.9836 - val_acc: 0.6953
Epoch 9/15
 - 11s - loss: 0.4084 - acc: 0.8643 - val_loss: 1.0086 - val_acc: 0.6878
Epoch 10/15
 - 12s - loss: 0.3943 - acc: 0.8695 - val_loss: 1.0133 - val_acc: 0.6926
Epoch 00010: early stopping
14958/14958 [==============================] - 2s 131us/step
Validation Accuracy: 69.2606%
Validation Loss: 1.0133354874335998
Test Accuracy: 0.8117543084401238
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'lr': 1e-07, 'beta_2': 0.999, 'beta_1': 0.3}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 23s - loss: 1.8442 - acc: 0.1671 - val_loss: 1.7846 - val_acc: 0.2191
Epoch 2/15
 - 13s - loss: 1.8272 - acc